Train a Tokonizer


In [1]:
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
from accelerate import Accelerator
import os
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("GPU is available and being used.")
else:
    device = torch.device('cpu')
    print("GPU is not available, using CPU.")

2024-07-20 11:48:57.011815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 11:48:57.011929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 11:48:57.129591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
#import shutil

# Remove the folder and its contents
#shutil.rmtree('/kaggle/working/diseaseBert')


# diseaseBert1_60_epochs_with_old_tokenizer

In [5]:

# Charger le tokenizer et le modèle BERT pour le MLM
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint).to(device)  

# Charger le fichier texte
additional_texts = []
with open('/kaggle/input/ccccccc/cleaned_concatenated_text_file.txt', 'r', encoding='utf-8') as file:  
    for line in file:
        additional_texts.append(line.strip())

# Convertir en Dataset
additional_dataset = Dataset.from_dict({"text": additional_texts})

# Tokeniser le dataset additionnel
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)  

tokenized_additional_dataset = additional_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Préparer le collateur de données pour MLM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)




training_args = TrainingArguments(
     output_dir="/kaggle/working/diseaseBert_old_token",
    overwrite_output_dir=True,
    num_train_epochs=60, 
    per_device_train_batch_size=8,  
    save_steps=8000,  # 
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=5e-5,  
    weight_decay=0.01,  
    fp16=True  
)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_additional_dataset,
)

# Entraîner le modèle
trainer.train()




GPU is available and being used.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,1.387200
1000,1.182500
1500,1.096100
2000,1.022900
2500,0.976000
3000,0.938700
3500,0.906000
4000,0.873700
4500,0.854000
5000,0.826200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=43260, training_loss=0.5138764556447389, metrics={'train_runtime': 15021.5046, 'train_samples_per_second': 46.022, 'train_steps_per_second': 2.88, 'total_flos': 4.5489687796224e+16, 'train_loss': 0.5138764556447389, 'epoch': 60.0})

In [6]:
!pip install huggingface_hub
from huggingface_hub import notebook_login


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
from huggingface_hub import notebook_login

notebook_login()


In [10]:
trainer.push_to_hub()

events.out.tfevents.1721476251.a2ca2d850ec6.34.0:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jaafer/diseaseBert_old_token/commit/d4747135f292ea7cf838062cfa0ec75717560187', commit_message='End of training', commit_description='', oid='d4747135f292ea7cf838062cfa0ec75717560187', pr_url=None, pr_revision=None, pr_num=None)